If you want to use this notebook online without installing Python on your computer, try:
<a href="https://colab.research.google.com/github/WetSuiteLeiden/example-notebooks/blob/main/wetsuite-nlp-crash-course/3-a-first-nlp-experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Google Colab"/></a> (do note however that this requires a Google account).

# WetSuite NLP crash course
# Part 3: A first NLP experiment

In part 3 of this crash course, we will finally build our first NLP experiment! This third part is a simplified version of the quantitative experiment done in Martijn Staal's bachelor thesis [_De memorie van toelichting overvraagd?_](https://martijn-staal.nl/nl/blog/de-memorie-van-toelichting-overvraagd). In this thesis, Martijn wanted to research how parliamentary documents (the _Kamerstukken_) are actually used in Dutch legal practice. One way to research this is to analyze how often such documents are cited in legal documents. The method he employed was to automatically search for such citations in legal cases and legal literature.

In this part of the crash course we will recreate a simplified version of this experiment, and try to count any type of citation to the Kamerstukken in Dutch legal cases.

# Sources
This notebook is based on Martijn Staal's bachelor thesis [_De memorie van toelichting overvraagd?_](https://martijn-staal.nl/nl/blog/de-memorie-van-toelichting-overvraagd) and the accompanying [digital appendix](https://github.com/mastaal/de-mvt-overvraagd).

# Step 1: Pre-processing our data
Before we can analyze anything, we of course need to get access to some data! Luckily for us, all the published court decisions in the Netherlands are also available via an online API (See ["Open Data van de Rechtspraak"](https://www.rechtspraak.nl/Uitspraken/Paginas/Open-Data.aspx)). This means that it is relatively straightforward to collect court decisions to analyze. To make your our first experiment even easier, we will use the WetSuite sample dataset `rechtspraak-sample-xml`. WetSuite has already downloaded some of the court decisions and made them available in an easily accessible bundle to make this first experiment easier.

Because we want to count citations to the Kamerstukken in these legal cases, we will need to find just the text of each decision to test it for such citations.

So, just like in part 2, we start with installing the wetsuite library and importing it:

In [ ]:
!pip3 install -U wetsuite
import wetsuite.datasets

Then we can download the dataset (if you've already downloaded it before and it's still on your PC, it will just load the dataset without re-downloading it):

In [7]:
# rd, short for rechtspraak-dataset
# Note that this download can take a while, as it is about 450 MB.
# Make sure you have enough space available on your device, as the uncompressed
# dataset is nearly 4GB.
rd = wetsuite.datasets.load("rechtspraaknl-sample-xml")

Now, let us see in what format the data is that we get from the Open Data Rechtspraak API:

In [ ]:
# Get a random key and value from the dataset
key = "https://data.rechtspraak.nl/uitspraken/content?id=ECLI:NL:CBB:2022:1"
value = rd.data.get(key)

# This is a bit of a technical step, which you can skip for now.
# But if you want to know what it does: The value is stored as a bitstring,
# but to print it readably we decode it to a proper UTF-8 string. UTF-8 is
# a way to encode human-readable characters in binary zeroes and ones.
value_str: str = value.decode()

print(value_str)

Well, that is clearly not just the text of the decision. What's with all the `<>`-brackets? It turns out the Open Data Rechtspraak API makes the court decisions available in the machine-readable [XML](https://en.wikipedia.org/wiki/XML)-format. XML is together with [JSON](https://en.wikipedia.org/wiki/JSON) one of the major two general formats for computers to exchange information in a machine-readable way. Many more specific file standards are based on XML, such as HTML.

Generally the provider of an API also provides documentation on the _structure_ of the XML (or JSON) that is send by the API. The Dutch Judicial Council does this as well (although it is only available in Dutch): [Open Data van de Rechtspraak, versie 1.15](https://www.rechtspraak.nl/SiteCollectionDocuments/Technische-documentatie-Open-Data-van-de-Rechtspraak.pdf). Here, starting at page 16, it is exactly defined how the XML that we have is structured and what we can find where. The XML for each decision consists of metadata and the actual text with paragraph structures.

In our simple experiment, we just want to count the number of citations per decision, so we can ignore the metadata. In a real experiment, you probably do not want to do that, since the metadata can help you distinguish your results better. For example: maybe specific (types of) courts cite more often to parliamentary documents, or there is a specific time frame in which this occurs more often. To research such more specific things, you will need the metadata.

According to the specification, we can find the text of the decision in the `<uitspraak></uitspraak>` elements. But what even is an XML element? As a very short introduction into XML: in an XML file, XML elements are structured in a hierarchy. Each element has a start-tag and end-tag with the same name, and may have attributes and/or content.

For example, consider this - shortened - XML document which is provided by the API of the Judicial Council:
```XML
<?xml version="1.0" encoding="utf-8"?>
<open-rechtspraak>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:ecli="https://e-justice.europa.eu/ecli" xmlns:tr="http://tuchtrecht.overheid.nl/" xmlns:eu="http://publications.europa.eu/celex/" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:bwb="bwb-dl" xmlns:cvdr="http://decentrale.regelgeving.overheid.nl/cvdr/" xmlns:psi="http://psi.rechtspraak.nl/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#">
    <rdf:Description>
      <dcterms:creator rdfs:label="Instantie" resourceIdentifier="http://standaarden.overheid.nl/owms/terms/College_van_Beroep_voor_het_bedrijfsleven" scheme="overheid.RechterlijkeMacht">College van Beroep voor het bedrijfsleven</dcterms:creator>
      <dcterms:date rdfs:label="Uitspraakdatum">2022-01-11</dcterms:date>
      <psi:zaaknummer rdfs:label="Zaaknr">20/1063</psi:zaaknummer>
    </rdf:Description>
    <rdf:Description rdf:about="http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:CBB:2022:1">
      <dcterms:title rdf:language="nl">ECLI:NL:CBB:2022:1 College van Beroep voor het bedrijfsleven , 11-01-2022 / 20/1063</dcterms:title>
    </rdf:Description>
  </rdf:RDF>
  <uitspraak id="ECLI:NL:CBB:2022:1:DOC" lang="nl" xml:space="preserve" xmlns="http://www.rechtspraak.nl/schema/rechtspraak-1.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xlink="http://www.w3.org/1999/xlink">
  <section role="overwegingen">
    <title>Overwegingen </title>
    <paragroup>
      <para>6. Het beroep is gegrond en het College vernietigt het bestreden besluit. Het College ziet geen aanleiding zelf in de zaak te voorzien, omdat het aan verweerder is om op de aanvraag te beslissen en de hoogte van de subsidie vast te stellen. Verweerder zal daarom een nieuw besluit moeten nemen met inachtneming van deze uitspraak. Het College stelt hiervoor een termijn van zes weken. 	</para>
      <para />
      <para>7. Voor een veroordeling in de proceskosten is geen aanleiding, omdat niet is gebleken van te vergoeden proceskosten.</para>
    </paragroup>
  </section>
  <section role="beslissing">
    <title>Beslissing</title>
    <parablock>
      <para>Het College:</para>
    </parablock>
    <para>- verklaart het beroep gegrond;</para>
  </section>
  </uitspraak>
</open-rechtspraak>
```

So we can see that for example `<para>Het College:</para>` is an element with content "Het College:" and no attributes, and that this element is hierarchically below the `<parablock>...</parablock>` element above it. Generally, we will say that the `para` element is a _child_ of the `parablock` element. This latter element is then the _parent_ of its _child_. We can see in this small overview also that the complete decision is located in the `<uitspraak>...</uitspraak>` element. If we want the text of the decision, we thus need to find this element. If you want to know exactly which content you can expect where in the XML documents provided by an API, be sure to check the API documentation.

Python has a built-in [XML library](https://docs.python.org/3/library/xml.etree.elementtree.html#tutorial) which we can use to select just the part we need. So first, let's import the XML library and define the required namespaces:

In [9]:
# Import the XML library
# Note that normally you prefer to put all the inputs at the top of your Python file.
import xml.etree.ElementTree as ET

# Define the XML namespaces that we can expect in our XML
# See for more information: https://en.wikipedia.org/wiki/XML_namespace
XML_NS = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "dcterms": "http://purl.org/dc/terms/",
    "psi": "http://psi.rechtspraak.nl/",
    "rs": "http://www.rechtspraak.nl/schema/rechtspraak-1.0",
    "ecli": "https://e-justice.europa.eu/ecli"
}

Now we can load the XML data:

In [ ]:
value_xml = ET.fromstring(value)

print(value_xml)

Now we can use Python's [xml library to find the appropriate XML element](https://docs.python.org/3/library/xml.etree.elementtree.html#finding-interesting-elements), which was `<uitspraak></uitspraak>` according to the documentation. Since the namespace for this element is `http://www.rechtspraak.nl/schema/rechtspraak-1.0`, we can use the shorthand `rs` we defined in our `XML_NS` dictionary above. So we need to look for the `rs:uitspraak` element:

In [ ]:
uitspraakxml = value_xml.find("rs:uitspraak", XML_NS)

print(uitspraakxml)

Okay, neat. We've found the correct element. But we want to search through the text within the element, preferably without all the XML things. Luckily, Python provides the [xml.etree.ElementTree.tostring](https://docs.python.org/3/library/xml.etree.elementtree.html#xml.etree.ElementTree.tostring) function, which we can use to convert the content of a parsed XML element into just plain text:

In [ ]:
uitspraak_bitstring = ET.tostring(uitspraakxml, encoding="utf-8", method="text")
# Since XML is a binary format, this results in a bitstring, so we need to decode it to get a proper string:
uitspraak_str = uitspraak_bitstring.decode()

print(uitspraak_str)

Now we have managed to convert this one court decision into some neat plaintext, in which we can search for our citations. However, we've only preprocessed this one decision. We will probably want to do that to a lot more decisions. So let us convert this procedure into a re-usable function.

In [ ]:
def preprocess_decision(rd: wetsuite.datasets.Dataset, key: str) -> str:
    """Retrieve the value corresponding to the given key and return the decision text in plaintext"""

    value = rd.data.get(key)

    return preprocess_raw_value(value)

def preprocess_raw_value(value) -> str:

    value_xml = ET.fromstring(value)
    
    uitspraakxml = value_xml.find("rs:uitspraak", XML_NS)

    if uitspraakxml is None:
        # It was not possible to find an uitspraak tag, returning error string.
        # Raising a custom Exception or Error would be more neat, but this works for a small example as this.
        return "!!!ERROR: NO DECISION FOUND!!!"

    uitspraak_bitstring = ET.tostring(uitspraakxml, encoding="utf-8", method="text")
    # Since XML is a binary format, this results in a bitstring, so we need to decode it to get a proper string:
    uitspraak_str = uitspraak_bitstring.decode()

    return uitspraak_str

We can now simply call this function with any key to do our pre-processing:

In [ ]:
print(preprocess_decision(rd, "https://data.rechtspraak.nl/uitspraken/content?id=ECLI:NL:RVS:2022:1"))


# Step 2: Searching for citations

We want to know how many citations there are to Kamerstukken in a court decision. Some examples of such citations are:

```
Kamerstukken II 2022/23, 36 229, nr. 1
Kamerstukken II 2020/21, 35 791, nr. 3
Kamerstukken II 2005/06, 30 316, nr. 3, p. 7–8
Kamerstukken II 2021/22, 29 628, nr. 1051
Kamerstukken II 2019/20, 35 300-XV, nr. 28
Kamerstukken I 2003/04, 27 484 (R 1669), nr. 289
Kamerstukken I 2020/21, 35 570, C
Kamerstukken I 2021/22, 35 925, nr. E
```

So how can we look for them? The first thing that comes to mind is to search for what the examples all have in common: they start with _Kamerstukken_. The simplest way to test if some string is contained in a longer string is using `in`:

In [ ]:
print("Kamerstukken" in "False test")
print("Kamerstukken" in "Kamerstukken I 2021/22, 35 925, nr. E")

However, this only gives a binary result. When there are multiple citations in the same text, it will say _True_ all the same. We need something better to find patterns in text.

One approach to this is to use [_regular expressions_](https://en.wikipedia.org/wiki/Regular_expression) (often abbreviated to just _regexp_ or _regex_). A regex is a description of some textual pattern. In many programming languages, such regexes can be used to search for the patterns which they describe. In Python, the [re](https://docs.python.org/3/library/re.html) library provides this functionality.

For a more thorough introduction to regexes, there's a free Codecademy course: [Learn the Basics of Regular Expressions](https://www.codecademy.com/learn/introduction-to-regular-expressions), and also [RegexOne's interactive online course](https://www.regexone.com/).

If we just want to count citations, we can start out with a rough pattern that is just `Kamerstukken`.

In [ ]:
import re

regex_pattern = r"Kamerstukken"

sample_text = preprocess_decision(rd, "https://data.rechtspraak.nl/uitspraken/content?id=ECLI:NL:HR:2022:394")

re.findall(regex_pattern, sample_text)

Great, it seems that we've found two citations! But, our pattern was quite rough. We need to take into account that our pattern might also match _other_ snippets of text that are not the citations we are looking for. For now, this is enough. However, if you want you can try to make a more specific pattern which will _only_ match citations.

## A short detour: looking for more complex patterns
Depending on what the goals are of your research you might need a more precise pattern, or a rough approach might be okay. Although regular expressions can technically be used to also find more elaborate and complex patterns, there is a point at which they become unmanageable. If the textual patterns you are looking for reach this point, then there are generally two possible approaches.

The first approach for finding (and making sense of) complex textual patterns is to use a _parser_ for the pattern you are looking for. A [parser](https://en.wikipedia.org/wiki/Parsing#Parser) is a piece of software that reads some input text and converts it to a formal structure as defined by a certain _grammar_. The grammar is the definition of your textual pattern. Existing parsing libraries can then be used to generate a parser based on your grammar. [Lark](https://github.com/lark-parser/lark) is for example a parsing toolkit for use with Python. With a grammar, you can more easily describe complex patterns. [Here is an example of a Lark grammar for Dutch parliamentary citations](https://github.com/mastaal/nllegalcit/blob/main/nllegalcit/grammars/parlementair.lark) (which is - even with nearly a hundred lines - still not complete for all possible citations!). Another useful features of parsers is that you can also easily extract information from the text snippets that matched your pattern. This is what [the nllegalcit library uses to extract specific information about a citation to the Kamerstukken](https://nllegalcit.martijn-staal.nl/latest/nllegalcit.html#nllegalcit.KamerstukCitation).

The second approach is to use tools based on machine learning to find the textual snippets you are looking for. For this you will generally need quite an expansive dataset of annotated examples so that you can train a model to accurately recognize the textual pattern. This approach might be useful when it is hard to precisely define what textual pattern you are looking for.

We will leave this now for another time, and for now settle for our basic regex. Before we're completely done with this step, let's make a simple function that given some text returns the number of matches of our regex:

In [ ]:
def get_citation_counts(text: str) -> int:
    regex_pattern = r"Kamerstukken"

    matches = re.findall(regex_pattern, text)

    return len(matches)

# Step 3: Getting our statistics

We now have the two elementary parts that we need to put together for our experiment: our preprocessing step, and our function to get results for a single piece of text. If we run these two steps for a lot of items in our dataset, we can get some statistics that might be interesting!



In [ ]:
# How many randomly sampled decisions will we analyze?
n = 50

# A list of key-value tuples
random_samples = rd.data.random_sample(n)

results = {}

for key, value in random_samples:
    # Preprocess to get the text
    text = preprocess_raw_value(value)

    # Find the citations
    citation_count = get_citation_counts(text)
    print(key, citation_count)

    # Add the results to our dictionary
    results[key] = citation_count

Since it randomly samples a new set of `n` decisions, try and run it a few times and see how the results change.

Now that we have our results dictionary, we can generate some statistics:

In [ ]:
# The python statistics library has more built in functions: https://docs.python.org/3/library/statistics.html
# and the third-party NumPy and SciPy libraries have even more elaborate tools for scientific statistics.
import statistics

print("Average:", statistics.median(results.values()))
print("Population standard deviation:", statistics.pstdev(results.values()))

Be sure to try out and see how the results change if you change the `n`. It seems that it is generally rare that a court decision cites a Kamerstuk. However, it would obviously be more interesting to analyze _all_ decisions in the dataset, and to split them out per court type. Go ahead and try out to add new features and make the experiment a bit more insightful!

# Done!